In [15]:
import os
import numpy as np
from numpy import asarray
import PIL
from PIL import Image
import skimage
from skimage.transform import resize
from skimage.color import rgb2gray,gray2rgb
from skimage.filters import median,threshold_otsu,meijering
from skimage.exposure import equalize_adapthist
from matplotlib import pyplot as plt
from skimage.io import imread

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
DIR="/content/drive/MyDrive/Colab Notebooks/images"
IMG_SIZE = 250
X= []*3838
y=[]

In [ ]:
def preprocess(img):
  img = rgb2gray(img)
  img=meijering(img)
  img = gray2rgb(img)
  return img

In [ ]:
for category in os.listdir(DIR):
    path = os.path.join(DIR, category)
    for img in os.listdir(path):
        class_no = os.listdir(DIR).index(category)
        filename = os.path.join(path, img)
        img_array = asarray(Image.open(filename).convert('RGB'))
        img_array = resize(img_array, (250, 250))
        plt.imsave(filename,img_array)

In [ ]:
for category in os.listdir(DIR):
    path = os.path.join(DIR, category)
    for img in os.listdir(path):
        class_no = os.listdir(DIR).index(category)
        filename = os.path.join(path, img)
        img_array = asarray(Image.open(filename).convert('RGB'))
        augmented_array=preprocess(img_array)
        filename=filename[:-4] + '-augmented.jpg'
        plt.imsave(filename,augmented_array)

In [8]:
i=0
for category in os.listdir(DIR):
    path = os.path.join(DIR, category)
    for img in os.listdir(path):
      i+=1

print("amount of images= "+str(i))

amount of images= 3838


In [17]:
for category in os.listdir(DIR):
    path = os.path.join(DIR, category)
    for img in os.listdir(path):
        class_no = os.listdir(DIR).index(category)
        filename = os.path.join(path, img)
        img_array = asarray(skimage.io.imread(filename))
        X.append(img_array)
        y.append(class_no)

In [ ]:
X=np.array(X).reshape(-1,IMG_SIZE,IMG_SIZE,3)
y = np.array(y)
X=X/255.0

In [ ]:
import tensorflow as tf
from keras_preprocessing.image import ImageDataGenerator
import sklearn
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33,shuffle= True)
X_train, X_val, y_train, y_val  = train_test_split(X_train, y_train, test_size=0.33, shuffle= True )

In [ ]:
base_model = InceptionV3(
                                weights='imagenet',
                                include_top=False,
                                input_shape=(250, 250,3)
                                )

base_model.trainable=False

model = tf.keras.Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(10, activation='softmax'))

87916544/87910968 [==============================] - 1s 0us/step


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Functional)    (None, 6, 6, 2048)        21802784  
_________________________________________________________________
flatten (Flatten)            (None, 73728)             0         
_________________________________________________________________
dense (Dense)                (None, 10)                737290    
Total params: 22,540,074
Trainable params: 737,290
Non-trainable params: 21,802,784
_________________________________________________________________


In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer='adam', metrics=['accuracy'])

In [ ]:
aug = ImageDataGenerator()
model.fit(aug.flow(X_train, y_train),validation_data = (X_val, y_val),epochs = 15,verbose=2)

Epoch 1/15
23/23 - 4s - loss: 4.1168 - accuracy: 0.1195 - val_loss: 2.9613 - val_accuracy: 0.1333
Epoch 2/15
23/23 - 2s - loss: 2.4298 - accuracy: 0.1717 - val_loss: 2.1946 - val_accuracy: 0.2583
Epoch 3/15
23/23 - 2s - loss: 2.0768 - accuracy: 0.2720 - val_loss: 2.2642 - val_accuracy: 0.2472
Epoch 4/15
23/23 - 2s - loss: 2.0020 - accuracy: 0.3173 - val_loss: 2.1984 - val_accuracy: 0.2333
Epoch 5/15
23/23 - 2s - loss: 1.9930 - accuracy: 0.3063 - val_loss: 2.1839 - val_accuracy: 0.2472
Epoch 6/15
23/23 - 2s - loss: 1.8542 - accuracy: 0.3874 - val_loss: 2.2202 - val_accuracy: 0.2750
Epoch 7/15
23/23 - 2s - loss: 1.8843 - accuracy: 0.3338 - val_loss: 2.4466 - val_accuracy: 0.1944
Epoch 8/15
23/23 - 2s - loss: 1.8405 - accuracy: 0.3846 - val_loss: 2.3286 - val_accuracy: 0.2806
Epoch 9/15
23/23 - 2s - loss: 1.8072 - accuracy: 0.4286 - val_loss: 2.3664 - val_accuracy: 0.2278
Epoch 10/15
23/23 - 2s - loss: 1.6567 - accuracy: 0.4451 - val_loss: 2.2408 - val_accuracy: 0.2917
Epoch 11/15
23/23 -

In [ ]:
model.save("dog_classifier.model")

In [ ]:
the_model=tf.keras.models.load_model("dog_classifier.model")